Before we can use the rest of the notebook, we need to install the dependencies: this example uses transformers. To use TPUs on Colab, we need to install torch_xla and the last line installs accelerate from source.

# Import of Libraries

In [ ]:
!pip --version

pip 19.3.1 from /usr/local/lib/python3.7/dist-packages/pip (python 3.7)


Make sure that we connect to TPU.

In [ ]:
import os
assert os.environ['COLAB_TPU_ADDR'], 'Make sure to select TPU from Edit > Notebook settings > Hardware accelerator'

We need to install 🤗 Accelerate in a virtual environment. 

In [ ]:
! pip install virtualenv

     |████████████████████████████████| 7.2MB 5.0MB/s 
     |████████████████████████████████| 348kB 38.0MB/s 


Check virtual environment version.

In [ ]:
!virtualenv --version

virtualenv 20.4.7 from /usr/local/lib/python3.7/dist-packages/virtualenv/__init__.py


In [ ]:
!virtualenv my_project

created virtual environment CPython3.7.10.final.0-64 in 733ms
  creator CPython3Posix(dest=/content/my_project, clear=False, no_vcs_ignore=False, global=False)
  seeder FromAppData(download=False, pip=bundle, setuptools=bundle, wheel=bundle, via=copy, app_data_dir=/root/.local/share/virtualenv)
    added seed packages: pip==21.1.2, setuptools==57.0.0, wheel==0.36.2
  activators BashActivator,CShellActivator,FishActivator,PowerShellActivator,PythonActivator,XonshActivator


In [ ]:
!source my_project/bin/activate

Installing datasets and transfomers as well as Accelerate.

In [ ]:
! pip install datasets transformers
! pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl
# ! pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.8.1-cp37-cp37m-linux_x86_64.whl
! pip install git+https://github.com/huggingface/accelerate
# ! pip install accelerate

     |████████████████████████████████| 266kB 5.2MB/s 
     |████████████████████████████████| 2.5MB 10.0MB/s 
     |████████████████████████████████| 122kB 38.9MB/s 
     |████████████████████████████████| 245kB 41.7MB/s 
     |████████████████████████████████| 901kB 23.2MB/s 
     |████████████████████████████████| 3.3MB 39.9MB/s 
ERROR: transformers 4.8.2 has requirement huggingface-hub==0.0.12, but you'll have huggingface-hub 0.0.13 which is incompatible.
     |████████████████████████████████| 149.9MB 78kB/s 
     |████████████████████████████████| 61kB 2.9MB/s 
ERROR: earthengine-api 0.1.272 has requirement google-api-python-client<2,>=1.12.1, but you'll have google-api-python-client 1.8.0 which is incompatible.
  Found existing installation: google-api-python-client 1.12.8
    Uninstalling google-api-python-client-1.12.8:
      Successfully uninstalled google-api-python-client-1.12.8
  Cloning https://github.com/huggingface/accelerate to /tmp/pip-req-build-fey8mtxm
  Running com

To check transformers is properly installed, we run the following command:

In [ ]:
from transformers import pipeline
print(pipeline('sentiment-analysis')('we love you'))


[{'label': 'POSITIVE', 'score': 0.9998704791069031}]


Here are all the imports we will need for this notebook.

In [ ]:
import torch
from torch.utils.data import DataLoader, TensorDataset, RandomSampler

from accelerate import Accelerator, DistributedType
from datasets import load_dataset, load_metric
from transformers import (
    AdamW,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    get_linear_schedule_with_warmup,
    set_seed,
)

from tqdm.auto import tqdm



# Import of Data

Download data from Kaggle 

In [ ]:
import os

# Download Kaggle Dataset via the Kaggle API

kaggle_api = {"username":"aygulzagidullina","key":"da122ee89f2edede46cc7dc5d55d55ba"}

import json
with open('/content/kaggle.json', 'w') as file:
    json.dump(kaggle_api, file)

!chmod 600 /content/kaggle.json
os.environ['KAGGLE_CONFIG_DIR'] = "/content"

!kaggle competitions download -c jigsaw-unintended-bias-in-toxicity-classification

!unzip \*.zip  && rm *.zip

 95% 311M/326M [00:02<00:00, 155MB/s]
100% 326M/326M [00:02<00:00, 156MB/s]
 91% 11.0M/12.1M [00:00<00:00, 115MB/s]
100% 12.1M/12.1M [00:00<00:00, 107MB/s]
 99% 64.0M/64.7M [00:00<00:00, 141MB/s]
100% 64.7M/64.7M [00:00<00:00, 163MB/s]
  0% 0.00/224k [00:00<?, ?B/s]
100% 224k/224k [00:00<00:00, 71.2MB/s]
 57% 9.00M/15.9M [00:00<00:00, 90.1MB/s]
100% 15.9M/15.9M [00:00<00:00, 101MB/s] 
 76% 12.0M/15.8M [00:00<00:00, 125MB/s]
100% 15.8M/15.8M [00:00<00:00, 126MB/s]
 41% 5.00M/12.3M [00:00<00:00, 47.1MB/s]
100% 12.3M/12.3M [00:00<00:00, 54.0MB/s]
 97% 268M/276M [00:01<00:00, 162MB/s]
100% 276M/276M [00:01<00:00, 153MB/s]
Archive:  all_data.csv.zip
  inflating: all_data.csv            

Archive:  sample_submission.csv.zip
  inflating: sample_submission.csv   

Archive:  train.csv.zip
  inflating: train.csv               

Archive:  test.csv.zip
  inflating: test.csv                

Archive:  test_public_expanded.csv.zip
  inflating: test_public_expanded.csv  

Archive:  test_private_expan

Load train dataset, split with 10% for the validation in the training procedure.

In [ ]:
from datasets import load_dataset

dataset_dict_train = load_dataset('csv', data_files='/content/train.csv')
dataset_train = dataset_dict_train['train']


Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-5ec55c99d77ea78c/0.0.0/e138af468cb14e747fb46a19c787ffcfa5170c821476d20d5304287ce12bbc23. Subsequent calls will reuse this data.


*Select* only 600 000 rows

In [ ]:
 # choosing only part of the data
 
 my_list = range(600000) 
 dataset_train = dataset_train.select(my_list)

 dataset_train = dataset_train.train_test_split(test_size=0.1) 

Create copy of the column 'target', i.e. 'labels'

In [ ]:
dataset_train = dataset_train.map(lambda batch: {'labels': batch['target']}, batched = True)

In [ ]:
dataset_train = dataset_train.map(lambda example: {'labels': 1 if example['labels'] >= 0.5 else 0})

Change the float64 type of 'labels' column to int64

In [ ]:
from datasets import ClassLabel, Value

new_features = dataset_train['train'].features.copy()

new_features["labels"] = Value('int64') 

dataset_train['train'] = dataset_train['train'].cast(new_features)

Change the int64 type to ClassLabel for 'labels' column

In [ ]:
from datasets import ClassLabel, Value

new_features = dataset_train['train'].features.copy()

new_features["labels"] = ClassLabel(names=['non toxic', 'toxic'])  

dataset_train['train'] = dataset_train['train'].cast(new_features)

Function to print 2 examples of the data

In [ ]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=2):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))


In [ ]:
show_random_elements(dataset_train["train"])

,article_id,asian,atheist,bisexual,black,buddhist,christian,comment_text,created_date,disagree,female,funny,heterosexual,hindu,homosexual_gay_or_lesbian,id,identity_annotator_count,identity_attack,insult,intellectual_or_learning_disability,jewish,labels,latino,likes,male,muslim,obscene,other_disability,other_gender,other_race_or_ethnicity,other_religion,other_sexual_orientation,parent_id,physical_disability,psychiatric_or_mental_illness,publication_id,rating,sad,severe_toxicity,sexual_explicit,target,threat,toxicity_annotator_count,transgender,white,wow
0,163965,None,None,None,None,None,None,"Trump has already been a disaster for Canada and the rest of the world by proclaiming a dirty climate change Denier regime; and by spewing his poisonous effluent and lies to corrupt democracy and to give support and energy to the neo nazis and fascists in Canada. \n\nGet a brain, Keller.",2017-01-25 15:19:28.796106+00,0,None,0,None,None,None,895300,0,0.057143,0.585714,None,None,toxic,None,0,None,None,0.014286,None,None,None,None,None,NaN,None,None,54,rejected,0,0.042857,0.0,0.614286,0.0,70,None,None,0
1,156431,None,None,None,None,None,None,"I didn't see anything written by Kristroff or supported by me that endorsed PE Trump. \n\nIt's clear that a President Trump wants to cut public service to the core, and in Republican history means all things are based on sustaining business profit. ""The rising tide lifts all boats... "" It's a theory. It's true in the sense that creating new products and services often provides broad benefits to those that consume those goods. It also sustains tax's that can be used to fund public projects. \n\nThe focus on wealth range can also result in amoral/immoral behaviors when profit temps human beings to compromise their values system. (Just call it 'gold fever' and most people get it.) \n\nWealth does NOT have to do this. There are many fine humanistic companies. But wealth/power can make a few people exceedingly dangerous to the public. It takes rules, boundaries, yes, regulation to prevent the extreme. Tight enough to restrain, but not so tight as to strangle",2016-12-15 01:29:32.798881+00,0,None,0,None,None,None,711568,0,0.000000,0.200000,None,None,non toxic,None,0,None,None,0.000000,None,None,None,None,None,707917.0,None,None,13,approved,0,0.000000,0.0,0.200000,0.0,5,None,None,0


Choose the tokenizer 

In [ ]:
from transformers import BertTokenizerFast
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

Function to clean text: remove stopwords, punctuation, lemmatize etc

In [ ]:
from bs4 import BeautifulSoup # Text Cleaning
import re, string # Regular Expressions, String
import nltk
from nltk.corpus import stopwords # stopwords
from nltk.stem.porter import PorterStemmer # for word stemming
from nltk.stem import WordNetLemmatizer # for word lemmatization
import unicodedata
import html

nltk.download('stopwords')
nltk.download('wordnet')

# set of stopwords to be removed from text
stop = set(stopwords.words('english'))

# update stopwords to have punctuation too
stop.update(list(string.punctuation))

def clean_text(text):
    
    # Remove unwanted html characters
    re1 = re.compile(r'  +')
    x1 = text.lower().replace('#39;', "'").replace('amp;', '&').replace('#146;', "'").replace(
    'nbsp;', ' ').replace('#36;', '$').replace('\\n', "\n").replace('quot;', "'").replace(
    '<br />', "\n").replace('\\"', '"').replace('<unk>', 'u_n').replace(' @.@ ', '.').replace(
    ' @-@ ', '-').replace('\\', ' \\ ')
    text = re1.sub(' ', html.unescape(x1))
    
    # remove non-ascii characters
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    
    # remove between square brackets
    text = re.sub('\[[^]]*\]', '', text)
    
    # remove URLs
    text = re.sub(r'http\S+', '', text)
    
    # remove twitter tags
    text = text.replace("@", "")
    
    # remove hashtags
    text = text.replace("#", "")
    
    # remove all non-alphabetic characters
    text = re.sub(r'[^a-zA-Z ]', '', text)
    
    # remove stopwords from text
    final_text = []
    for word in text.split():
        if word.strip().lower() not in stop:
            final_text.append(word.strip().lower())
    
    text = " ".join(final_text)
    
    # lemmatize words
    lemmatizer = WordNetLemmatizer()    
    text = " ".join([lemmatizer.lemmatize(word) for word in text.split()])
    text = " ".join([lemmatizer.lemmatize(word, pos = 'v') for word in text.split()])
    
    # replace all numbers with "num"
    text = re.sub("\d", "num", text)
    
    return text.lower()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


Apply "clean" function to the 'comment_text' column of the dataset

In [ ]:
dataset_train = dataset_train.map(lambda example: {'comment_text': clean_text(example['comment_text'])})

Create tokenized dataset

In [ ]:
tokenized_dataset = dataset_train.map(lambda batch: tokenizer(batch["comment_text"], truncation=True, padding="max_length", max_length=128), batched=True, \
                  remove_columns=['id', 'comment_text', 'severe_toxicity', 'obscene', 'identity_attack', 'insult', 'threat', 'asian', 'atheist', 'bisexual', 'black', \
                                  'buddhist', 'christian', 'female', 'heterosexual', 'hindu', 'homosexual_gay_or_lesbian', 'intellectual_or_learning_disability', \
                                  'jewish', 'latino', 'male', 'muslim', 'other_disability', 'other_gender', 'other_race_or_ethnicity', 'other_religion', 'other_sexual_orientation',\
                                  'physical_disability', 'psychiatric_or_mental_illness', 'target', 'transgender', 'white', 'created_date', \
                                  'publication_id', 'parent_id', 'article_id', 'rating', 'funny', 'wow', 'sad', 'likes', 'disagree', 'sexual_explicit',\
                                  'identity_annotator_count', 'toxicity_annotator_count'])

In [ ]:
tokenized_dataset["train"].features

{'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None),
 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'labels': ClassLabel(num_classes=2, names=['non toxic', 'toxic'], names_file=None, id=None),
 'token_type_ids': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None)}

Set the torch format for the dataset attributes

In [ ]:
tokenized_dataset.set_format("torch")

Choose the transformer model

In [ ]:
model_checkpoint = "bert-base-uncased"

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=2)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Downloading the testing data: Davidson et al 2017 data

In [ ]:
import pandas as pd

# Loading t-davidson raw dataset directly from the github

url = 'https://raw.githubusercontent.com/t-davidson/hate-speech-and-offensive-language/master/data/labeled_data.csv'
test_davidson = pd.read_csv(url)

from datasets import load_dataset
from datasets import Dataset

test_davidson = Dataset.from_pandas(test_davidson)

test_davidson = test_davidson.filter(lambda example: example['class'] != 1)

test_davidson = test_davidson.map(lambda example: {'labels': 1 if example['class'] == 0 else 0})

from datasets import ClassLabel, Value

new_features = test_davidson.features.copy()

new_features["labels"] = ClassLabel(names=['non toxic', 'toxic'])  

test_davidson = test_davidson.cast(new_features)

test_davidson = test_davidson.map(lambda example: {'tweet': clean_text(example['tweet'])})

In [ ]:
test_davidson = test_davidson.train_test_split(test_size=0.1) 

In [ ]:
test1_tokenized_dataset = test_davidson.map(lambda batch: tokenizer(batch["tweet"], truncation=True, padding="max_length", max_length=128), batched=True, \
                  remove_columns=['count', 'hate_speech', 'neither', 'offensive_language', 'Unnamed: 0', 'class', 'tweet'])

In [ ]:
test1_tokenized_dataset.set_format("torch")

Downloading the testing data: HASOC 2019

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving english_dataset.tsv to english_dataset.tsv


In [ ]:
import io

test_hasoc = pd.read_csv(io.BytesIO(uploaded['english_dataset.tsv']), sep='\t')

In [ ]:
test_hasoc = Dataset.from_pandas(test_hasoc)

test_hasoc = test_hasoc.filter(lambda example: example['task_2'] != 'PRFN' and example['task_2'] != 'OFFN')

test_hasoc = test_hasoc.map(lambda example: {'labels': 1 if example['task_2'] == 'HATE' else 0})

new_features = test_hasoc.features.copy()

new_features["labels"] = ClassLabel(names=['non toxic', 'toxic'])  

test_hasoc = test_hasoc.cast(new_features)

test_hasoc = test_hasoc.map(lambda example: {'text': clean_text(example['text'])})

In [ ]:
test_hasoc = test_hasoc.train_test_split(test_size=0.1) 

In [ ]:
test2_tokenized_dataset = test_hasoc.map(lambda batch: tokenizer(batch["text"], truncation=True, padding="max_length", max_length=128), batched=True, \
                  remove_columns=['task_1', 'task_2', 'task_3', 'text_id', 'text'])

In [ ]:
test2_tokenized_dataset.set_format("torch")

In [ ]:
def create_dataloaders(train_batch_size=8, eval_batch_size=8, test1_batch_size=8, test2_batch_size=8):
    train_dataloader = DataLoader(
        tokenized_dataset["train"], shuffle=True, batch_size=train_batch_size
    )
    eval_dataloader = DataLoader(
        tokenized_dataset["test"], shuffle=False, batch_size=eval_batch_size
    )
    test1_dataloader_train = DataLoader(
        test1_tokenized_dataset["train"], shuffle=True, batch_size=test1_batch_size
    )
    test1_dataloader_eval = DataLoader(
        test1_tokenized_dataset["test"], shuffle=False, batch_size=test1_batch_size
    )
    test2_dataloader_train = DataLoader(
        test2_tokenized_dataset["train"], shuffle=True, batch_size=test2_batch_size
    )
    test2_dataloader_eval = DataLoader(
        test2_tokenized_dataset["test"], shuffle=False, batch_size=test2_batch_size
    )
    return train_dataloader, eval_dataloader, test1_dataloader_train, test1_dataloader_eval, test2_dataloader_train, test2_dataloader_eval

In [ ]:
train_dataloader, eval_dataloader, test1_dataloader_test, test1_dataloader_eval, test2_dataloader_test, test2_dataloader_eval = create_dataloaders()

In [ ]:
for batch in train_dataloader:
    print({k: v.shape for k, v in batch.items()})
    outputs = model(**batch)
    break

{'attention_mask': torch.Size([8, 128]), 'input_ids': torch.Size([8, 128]), 'labels': torch.Size([8]), 'token_type_ids': torch.Size([8, 128])}


In [ ]:
outputs

SequenceClassifierOutput([('loss', tensor(0.7824, grad_fn=<NllLossBackward>)),
                          ('logits', tensor([[-0.3385, -0.1066],
                                   [-0.4104, -0.0700],
                                   [-0.3716, -0.1277],
                                   [-0.3955, -0.1695],
                                   [-0.4427, -0.2093],
                                   [-0.4801, -0.0139],
                                   [-0.4206, -0.0920],
                                   [-0.2709, -0.1487]], grad_fn=<AddmmBackward>))])

In [ ]:
# coding=utf-8
# Copyright 2020 The HuggingFace Datasets Authors and the current dataset script contributor.

%%file NewMetric.py

import datasets
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, roc_auc_score

# TODO: Add BibTeX citation
_CITATION = """  """

# TODO: Add description of the metric here
_DESCRIPTION = """  """


# TODO: Add description of the arguments of the metric here
_KWARGS_DESCRIPTION = """  """

@datasets.utils.file_utils.add_start_docstrings(_DESCRIPTION, _KWARGS_DESCRIPTION)
class NewMetric(datasets.Metric):
    """TODO: Short description of my metric."""

    def _info(self):
        # TODO: Specifies the datasets.MetricInfo object
        return datasets.MetricInfo(
            # This is the description that will appear on the metrics page.
            description=_DESCRIPTION,
            citation=_CITATION,
            inputs_description=_KWARGS_DESCRIPTION,
            # This defines the format of each prediction and reference
            features=datasets.Features({
                'predictions': datasets.Value('int64'),
                'references': datasets.Value('int64'),
            }),
      )

    def _compute(self, predictions, references):

        precision, recall, f1, _ = precision_recall_fscore_support(references, predictions, average='binary')
        acc = accuracy_score(references, predictions)

        try:
            auroc = roc_auc_score(references, predictions)
        except ValueError:
            pass
            auroc = 0.5

        return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall,
        'auroc': auroc
        } 

Writing NewMetric.py


In [ ]:
metric = load_metric('/content/NewMetric.py')

In [ ]:
predictions = outputs.logits.detach().argmax(dim=-1)
metric.compute(predictions=predictions, references=batch["labels"])

{'accuracy': 0.125,
 'auroc': 0.5,
 'f1': 0.2222222222222222,
 'precision': 0.125,
 'recall': 1.0}

In [ ]:
hyperparameters = {
    "learning_rate": 1e-5,
    "num_epochs": 3,
    "train_batch_size": 8, # Actual batch size will this x 8
    "eval_batch_size": 8, # Actual batch size will this x 8
    "test1_batch_size": 8, # Actual batch size will this x 8
    "test2_batch_size": 8, # Actual batch size will this x 8
    "seed": 42,
}

In [ ]:
import transformers

def training_function():
    # Initialize accelerator
    accelerator = Accelerator()

    # To have only one message (and not 8) per logs of Transformers or Datasets, we set the logging verbosity
    # to INFO for the main process only.
    if accelerator.is_main_process:
        datasets.utils.logging.set_verbosity_warning()
        transformers.utils.logging.set_verbosity_info()
    else:
        datasets.utils.logging.set_verbosity_error()
        transformers.utils.logging.set_verbosity_error()

    train_dataloader, eval_dataloader, test1_dataloader_train, test1_dataloader_eval, test2_dataloader_train, test2_dataloader_eval = create_dataloaders(
        train_batch_size=hyperparameters["train_batch_size"], eval_batch_size=hyperparameters["eval_batch_size"],
        test1_batch_size=hyperparameters["test1_batch_size"], test2_batch_size=hyperparameters["test2_batch_size"]
    )
    # The seed need to be set before we instantiate the model, as it will determine the random head.
    set_seed(hyperparameters["seed"])

    # Instantiate the model, let Accelerate handle the device placement.
    model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=2)

    # Instantiate optimizer
    optimizer = AdamW(params=model.parameters(), lr=hyperparameters["learning_rate"])

    # Prepare everything
    # There is no specific order to remember, we just need to unpack the objects in the same order we gave them to the
    # prepare method.
    model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
        model, optimizer, train_dataloader, eval_dataloader
    )

    num_epochs = hyperparameters["num_epochs"]
    # Instantiate learning rate scheduler after preparing the training dataloader as the prepare method
    # may change its length.
    lr_scheduler = get_linear_schedule_with_warmup(
        optimizer=optimizer,
        num_warmup_steps=500,
        num_training_steps=len(train_dataloader) * num_epochs,
    )

    # Instantiate a progress bar to keep track of training. Note that we only enable it on the main
    # process to avoid having 8 progress bars.
    progress_bar = tqdm(range(num_epochs * len(train_dataloader)), disable=not accelerator.is_main_process)

    # Now we train the model on Jigsaw data
    for epoch in range(num_epochs):
        model.train()
        for step, batch in enumerate(train_dataloader):
            outputs = model(**batch)
            loss = outputs.loss
            accelerator.backward(loss)
            
            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()
            progress_bar.update(1)

        model.eval()
        all_predictions = []
        all_labels = []

        for step, batch in enumerate(eval_dataloader):
            with torch.no_grad():
                outputs = model(**batch)
            predictions = outputs.logits.argmax(dim=-1)

            # We gather predictions and labels from the 8 TPUs to have them all.
            all_predictions.append(accelerator.gather(predictions))
            all_labels.append(accelerator.gather(batch["labels"]))

        # Concatenate all predictions and labels.
        # The last thing we need to do is to truncate the predictions and labels we concatenated
        # together as the prepared evaluation dataloader has a little bit more elements to make
        # batches of the same size on each process.
        all_predictions = torch.cat(all_predictions)[:len(tokenized_dataset["test"])]
        all_labels = torch.cat(all_labels)[:len(tokenized_dataset["test"])]

        eval_metric = metric.compute(predictions=all_predictions, references=all_labels)

        # Use accelerator.print to print only on the main process.
        accelerator.print(f"epoch {epoch}:", eval_metric)

###  transfer learning/training 

    optimizer = AdamW(params=model.parameters(), lr=hyperparameters["learning_rate"])
    
    model, optimizer, test1_dataloader_train, test1_dataloader_eval = accelerator.prepare(
        model, optimizer, test1_dataloader_train, test1_dataloader_eval
    )

    num_epochs = hyperparameters["num_epochs"]
    # Instantiate learning rate scheduler after preparing the training dataloader as the prepare method
    # may change its length.
    lr_scheduler = get_linear_schedule_with_warmup(
        optimizer=optimizer,
        num_warmup_steps=500,
        num_training_steps=len(test1_dataloader_train) * num_epochs,
    )

    progress_bar = tqdm(range(num_epochs * len(test1_dataloader_train)), disable=not accelerator.is_main_process)

    # Now we perform transfer learning/training on Davidson et al 2017 data

    for epoch in range(num_epochs):

        model.train()

        for param in model.bert.embeddings.parameters():
            param.requires_grad = False

        for step, batch in enumerate(test1_dataloader_train):
            outputs = model(**batch)
            loss = outputs.loss
            accelerator.backward(loss)
            
            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()
            progress_bar.update(1)

        model.eval()
        all_predictions = []
        all_labels = []

        for step, batch in enumerate(test1_dataloader_eval):
            with torch.no_grad():
                outputs = model(**batch)
            predictions = outputs.logits.argmax(dim=-1)

            # We gather predictions and labels from the 8 TPUs to have them all.
            all_predictions.append(accelerator.gather(predictions))
            all_labels.append(accelerator.gather(batch["labels"]))

        # Concatenate all predictions and labels.
        # The last thing we need to do is to truncate the predictions and labels we concatenated
        # together as the prepared evaluation dataloader has a little bit more elements to make
        # batches of the same size on each process.
        all_predictions = torch.cat(all_predictions)[:len(test1_tokenized_dataset["test"])]
        all_labels = torch.cat(all_labels)[:len(test1_tokenized_dataset["test"])]

        eval_metric = metric.compute(predictions=all_predictions, references=all_labels)

        # Use accelerator.print to print only on the main process.
        accelerator.print(f"Davidson et al 2017 data -- epoch {epoch}:", eval_metric)

######################################################################################################

    # Now we perform transfer learning/training on HASOC data

    optimizer = AdamW(params=model.parameters(), lr=hyperparameters["learning_rate"])

    model, optimizer, test2_dataloader_train, test2_dataloader_eval = accelerator.prepare(
        model, optimizer, test2_dataloader_train, test2_dataloader_eval
    )

    num_epochs = hyperparameters["num_epochs"]
    # Instantiate learning rate scheduler after preparing the training dataloader as the prepare method
    # may change its length.
    lr_scheduler = get_linear_schedule_with_warmup(
        optimizer=optimizer,
        num_warmup_steps=500,
        num_training_steps=len(test2_dataloader_train) * num_epochs,
    )

    progress_bar = tqdm(range(num_epochs * len(test2_dataloader_train)), disable=not accelerator.is_main_process)

    for epoch in range(num_epochs):

        model.train()

        for param in model.bert.embeddings.parameters():
            param.requires_grad = False

        for step, batch in enumerate(test2_dataloader_train):
            outputs = model(**batch)
            loss = outputs.loss
            accelerator.backward(loss)
            
            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()
            progress_bar.update(1)

        model.eval()
        all_predictions = []
        all_labels = []

        for step, batch in enumerate(test2_dataloader_eval):
            with torch.no_grad():
                outputs = model(**batch)
            predictions = outputs.logits.argmax(dim=-1)

            # We gather predictions and labels from the 8 TPUs to have them all.
            all_predictions.append(accelerator.gather(predictions))
            all_labels.append(accelerator.gather(batch["labels"]))

        # Concatenate all predictions and labels.
        # The last thing we need to do is to truncate the predictions and labels we concatenated
        # together as the prepared evaluation dataloader has a little bit more elements to make
        # batches of the same size on each process.
        all_predictions = torch.cat(all_predictions)[:len(test2_tokenized_dataset["test"])]
        all_labels = torch.cat(all_labels)[:len(test2_tokenized_dataset["test"])]

        eval_metric = metric.compute(predictions=all_predictions, references=all_labels)

        # Use accelerator.print to print only on the main process.
        accelerator.print(f"HASOC data -- epoch {epoch}:", eval_metric)


In [ ]:
%%time

from accelerate import notebook_launcher
 
notebook_launcher(training_function)

Launching a training on 8 TPU cores.


loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.8.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file https://huggingface.co/bert-base-uncased/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/tra

epoch 0: {'accuracy': 0.9485, 'f1': 0.6608122941822173, 'precision': 0.7131011608623549, 'recall': 0.6156678257312334, 'auroc': 0.7968469955532833}
epoch 1: {'accuracy': 0.9496166666666667, 'f1': 0.657838143746463, 'precision': 0.7364419665484034, 'recall': 0.5943955819185928, 'auroc': 0.7877622880651374}
epoch 2: {'accuracy': 0.9483166666666667, 'f1': 0.670071284179168, 'precision': 0.6982261640798226, 'recall': 0.6440989977500511, 'auroc': 0.8097016917221885}


Davidson et al 2017 data -- epoch 0: {'accuracy': 0.8875, 'f1': 0.7758007117437722, 'precision': 0.7841726618705036, 'recall': 0.7676056338028169, 'auroc': 0.8479176494372936}
Davidson et al 2017 data -- epoch 1: {'accuracy': 0.9142857142857143, 'f1': 0.8260869565217391, 'precision': 0.8507462686567164, 'recall': 0.8028169014084507, 'auroc': 0.8774850057281488}
Davidson et al 2017 data -- epoch 2: {'accuracy': 0.9267857142857143, 'f1': 0.8571428571428571, 'precision': 0.8482758620689655, 'recall': 0.8661971830985915, 'auroc': 0.9067828020756116}


HASOC data -- epoch 0: {'accuracy': 0.7130801687763713, 'f1': 0.2272727272727273, 'precision': 0.28169014084507044, 'recall': 0.19047619047619047, 'auroc': 0.5261324041811847}
HASOC data -- epoch 1: {'accuracy': 0.7679324894514767, 'f1': 0.20289855072463767, 'precision': 0.42424242424242425, 'recall': 0.13333333333333333, 'auroc': 0.5409214092140922}
HASOC data -- epoch 2: {'accuracy': 0.7784810126582279, 'f1': 0.14634146341463414, 'precision': 0.5, 'recall': 0.08571428571428572, 'auroc': 0.5306620209059233}
CPU times: user 2min 12s, sys: 21.1 s, total: 2min 33s
Wall time: 4h 27min 4s
